In [1]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [3]:
from pyspark.sql.types import IntegerType

In [12]:
df = sqlContext.read.csv('hdfs://master:9000/user/hadoop/taiwan_no1.csv', header=True)
df = df.withColumn('NotChinese', df['NotChinese'].cast(IntegerType()))
df.show()

+---+--------+---------------+---------+----------+
|_c0|    Dong|TotalPopulation|  Chinese|NotChinese|
+---+--------+---------------+---------+----------+
|  0|11110515|       13087712|  4103015|   8974948|
|  1|11110530|       35632256| 11148736|  24473864|
|  2|11110540|       15055109|  4224611|  10820654|
|  3|11110550|        5234450|  1060743|   4165353|
|  4|11110560|        2897171|   489187|   2401249|
|  5|11110570|         529710|   104003|    422920|
|  6|11110580|        1194300|   340557|    846654|
|  7|11110600|       11598836|  3484942|   8104366|
|  8|11110615|      123701130| 33203371|  90488065|
|  9|11110630|       19188805|  7957772|  11221359|
| 10|11110640|        6435391|  2432265|   3993554|
| 11|11110650|       12688271|  6454082|   6224392|
| 12|11110670|       17788427|  8207322|   9571383|
| 13|11110680|        3798949|  1926827|   1862249|
| 14|11110690|         308503|   156548|    147603|
| 15|11110700|        2980019|  1356400|   1614004|
| 16|1111071

In [13]:
maxValue = df.agg({'NotChinese': 'max'}).collect()[0][0]
maxValue

281292375

In [15]:
df_norm = df.withColumn('norm', df['NotChinese'] / maxValue)
df_norm = df_norm[['Dong', 'NotChinese', 'norm']]
df_norm.show()

+--------+----------+--------------------+
|    Dong|NotChinese|                norm|
+--------+----------+--------------------+
|11110515|   8974948| 0.03190611903362116|
|11110530|  24473864| 0.08700507434657623|
|11110540|  10820654|0.038467640653252685|
|11110550|   4165353|0.014807912941116871|
|11110560|   2401249| 0.00853648805802148|
|11110570|    422920|0.001503489029875...|
|11110580|    846654|0.003009871845975...|
|11110600|   8104366|0.028811182670699837|
|11110615|  90488065| 0.32168687473309576|
|11110630|  11221359| 0.03989215491532609|
|11110640|   3993554|0.014197164071724305|
|11110650|   6224392|  0.0221278376280196|
|11110670|   9571383| 0.03402645734709304|
|11110680|   1862249|0.006620332314375...|
|11110690|    147603|5.247316071045296E-4|
|11110700|   1614004|0.005737816391219279|
|11110710|   9467994| 0.03365890739128638|
|11140520|  89256370| 0.31730817445727066|
|11140540|  75261304|  0.2675554358698845|
|11140550| 281292375|                 1.0|
+--------+-

In [16]:
df_norm.repartition(1).write.csv('hdfs://master:9000/user/hadoop/new_foreigner_norm')